# Reinforcement Learning Space Invaders

In [1]:
# OpenGym CartPole-v0 with A3C on GPU
# -----------------------------------
#
# A3C implementation with GPU optimizer threads.
# 
# Made as part of blog series Let's make an A3C, available at
# https://jaromiru.com/2017/02/16/lets-make-an-a3c-theory/
#
# author: Jaromir Janisch, 2017

import numpy as np
import tensorflow as tf

import gym, time, random, threading

from keras.models import *
from keras.layers import *
from keras import backend as K

#-- constants
ENV = 'SpaceInvaders-ram-v4'

Reward = []
Reward_index = []
timer = 0
HOUR = 3600
RUN_TIME = HOUR*0.5
THREADS = 8
OPTIMIZERS = 2
THREAD_DELAY = 0.001

GAMMA = 0.99

N_STEP_RETURN = 8
GAMMA_N = GAMMA ** N_STEP_RETURN

EPS_START = 0.9
EPS_STOP  = .15
EPS_STEPS = RUN_TIME*0.95

MIN_BATCH = 32
LEARNING_RATE = 5e-3
LOSS_V = .5			# v loss coefficient
LOSS_ENTROPY = .01 	# entropy coefficient

#---------
class Brain:
    train_queue = [ [], [], [], [], [] ]	# s, a, r, s', s' terminal mask
    lock_queue = threading.Lock()

    def __init__(self):
        self.session = tf.Session()
        K.set_session(self.session)
        K.manual_variable_initialization(True)

        self.model = self._build_model()
        self.graph = self._build_graph(self.model)

        self.session.run(tf.global_variables_initializer())
        self.default_graph = tf.get_default_graph()

        self.default_graph.finalize()	# avoid modifications

    def _build_model(self):

        l_input = Input( batch_shape=(None, NUM_STATE) )
        l_dense = Dense(16, activation='relu')(l_input)

        out_actions = Dense(NUM_ACTIONS, activation='softmax')(l_dense)
        out_value   = Dense(1, activation='linear')(l_dense)

        model = Model(inputs=[l_input], outputs=[out_actions, out_value])
        model._make_predict_function()	# have to initialize before threading

        return model

    def _build_graph(self, model):
        s_t = tf.placeholder(tf.float32, shape=(None, NUM_STATE))
        a_t = tf.placeholder(tf.float32, shape=(None, NUM_ACTIONS))
        r_t = tf.placeholder(tf.float32, shape=(None, 1)) # not immediate, but discounted n step reward
        
        p, v = model(s_t)

        log_prob = tf.log( tf.reduce_sum(p * a_t, axis=1, keep_dims=True) + 1e-10)
        advantage = r_t - v

        loss_policy = - log_prob * tf.stop_gradient(advantage)									# maximize policy
        loss_value  = LOSS_V * tf.square(advantage)												# minimize value error
        entropy = LOSS_ENTROPY * tf.reduce_sum(p * tf.log(p + 1e-10), axis=1, keep_dims=True)	# maximize entropy (regularization)

        loss_total = tf.reduce_mean(loss_policy + loss_value + entropy)

        optimizer = tf.train.RMSPropOptimizer(LEARNING_RATE, decay=.99)
        minimize = optimizer.minimize(loss_total)

        return s_t, a_t, r_t, minimize

    def optimize(self):
        if len(self.train_queue[0]) < MIN_BATCH:
            time.sleep(0)	# yield
            return

        with self.lock_queue:
            if len(self.train_queue[0]) < MIN_BATCH:	# more thread could have passed without lock
                return 									# we can't yield inside lock

            s, a, r, s_, s_mask = self.train_queue
            self.train_queue = [ [], [], [], [], [] ]

        s = np.vstack(s)
        a = np.vstack(a)
        r = np.vstack(r)
        s_ = np.vstack(s_)
        s_mask = np.vstack(s_mask)

        if len(s) > 5*MIN_BATCH: print("Optimizer alert! Minimizing batch of %d" % len(s))

        v = self.predict_v(s_)
        r = r + GAMMA_N * v * s_mask	# set v to 0 where s_ is terminal state
        
        s_t, a_t, r_t, minimize = self.graph
        self.session.run(minimize, feed_dict={s_t: s, a_t: a, r_t: r})

    def train_push(self, s, a, r, s_):
        with self.lock_queue:
            self.train_queue[0].append(s)
            self.train_queue[1].append(a)
            self.train_queue[2].append(r)

            if s_ is None:
                self.train_queue[3].append(NONE_STATE)
                self.train_queue[4].append(0.)
            else:	
                self.train_queue[3].append(s_)
                self.train_queue[4].append(1.)

    def predict(self, s):
        with self.default_graph.as_default():
            p, v = self.model.predict(s)
            return p, v

    def predict_p(self, s):
        with self.default_graph.as_default():
            p, v = self.model.predict(s)		
            return p

    def predict_v(self, s):
        with self.default_graph.as_default():
            p, v = self.model.predict(s)		
            return v

#---------
frames = 0
class Agent:
    def __init__(self, eps_start, eps_end, eps_steps):
        self.eps_start = eps_start
        self.eps_end   = eps_end
        self.eps_steps = eps_steps

        self.memory = []	# used for n_step return
        self.R = 0.

    def getEpsilon(self):
        if(timer >= self.eps_steps):
            return self.eps_end
        else:
            return self.eps_start + timer * (self.eps_end - self.eps_start) / self.eps_steps	# linearly interpolate

    def act(self, s):
        eps = self.getEpsilon()			
        global frames; frames = frames + 1

        if random.random() < eps:
            return random.randint(0, NUM_ACTIONS-1)

        else:
            s = np.array([s])
            p = brain.predict_p(s)[0]

            # a = np.argmax(p)
            a = np.random.choice(NUM_ACTIONS, p=p)

            return a
    
    def train(self, s, a, r, s_):
        def get_sample(memory, n):
            s, a, _, _  = memory[0]
            _, _, _, s_ = memory[n-1]

            return s, a, self.R, s_

        a_cats = np.zeros(NUM_ACTIONS)	# turn action into one-hot representation
        a_cats[a] = 1 

        self.memory.append( (s, a_cats, r, s_) )

        self.R = ( self.R + r * GAMMA_N ) / GAMMA

        if s_ is None:
            while len(self.memory) > 0:
                n = len(self.memory)
                s, a, r, s_ = get_sample(self.memory, n)
                brain.train_push(s, a, r, s_)

                self.R = ( self.R - self.memory[0][2] ) / GAMMA
                self.memory.pop(0)		

            self.R = 0

        if len(self.memory) >= N_STEP_RETURN:
            s, a, r, s_ = get_sample(self.memory, N_STEP_RETURN)
            brain.train_push(s, a, r, s_)

            self.R = self.R - self.memory[0][2]
            self.memory.pop(0)	

    # possible edge case - if an episode ends in <N steps, the computation is incorrect

#---------
class Environment(threading.Thread):
    stop_signal = False

    def __init__(self, render=False, eps_start=EPS_START, eps_end=EPS_STOP, eps_steps=EPS_STEPS):
        threading.Thread.__init__(self)

        self.render = render
        self.env = gym.make(ENV)
        self.agent = Agent(eps_start, eps_end, eps_steps)

    def runEpisode(self):
        s = self.env.reset()

        R = 0
        while True:         
            time.sleep(THREAD_DELAY) # yield 

            if self.render: self.env.render()

            a = self.agent.act(s)
            s_, r, done, info = self.env.step(a)

            if done: # terminal state
                s_ = None

            self.agent.train(s, a, r, s_)

            s = s_
            R += r
            if done or self.stop_signal:
                break
        Reward.append(R)
        print("Total R:", R)

    def run(self):
        while not self.stop_signal:
            self.runEpisode()

    def stop(self):
        self.stop_signal = True

#---------
class Optimizer(threading.Thread):
    stop_signal = False

    def __init__(self):
        threading.Thread.__init__(self)

    def run(self):
        while not self.stop_signal:
            brain.optimize()

    def stop(self):
        self.stop_signal = True

#-- main
env_test = Environment(render=True, eps_start=0., eps_end=0.)
NUM_STATE = env_test.env.observation_space.shape[0]
NUM_ACTIONS = env_test.env.action_space.n
NONE_STATE = np.zeros(NUM_STATE)

brain = Brain()	# brain is global in A3C

envs = [Environment() for i in range(THREADS)]
opts = [Optimizer() for i in range(OPTIMIZERS)]

for o in opts:
    o.start()

for e in envs:
    e.start()

for i in range(0, int(RUN_TIME/10)):
    time.sleep(10)
    if not Reward:
        if not Reward_index:
            Reward_index.append(0)
        else:
            Reward_index.append(Reward_index[-1])
    else:
        avg_rew = 0
        avg_rew = sum(Reward)/len(Reward)
        Reward_index.append(avg_rew)
    Reward=[]
    timer += 10    
    print("Elapsed time(s): ", i*10)

for e in envs:
    e.stop()
for e in envs:
    e.join()

for o in opts:
    o.stop()
for o in opts:
    o.join()

#serializers.save_hdf5('test_save', brain.model)
#serializers.save_hdf5('test_save' + '.opt', opts)
#brain.model.save('test_save_mod.h5')
#opts.save('test_save_opt.h5')
print("Training finished")
#env_test.run()
                                               
                                               

Using TensorFlow backend.


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Total R: 65.0
Total R: 120.0
Total R: 30.0
Total R: 15.0
Elapsed time(s):  0
Total R: 125.0
Total R: 155.0
Total R: 260.0
Total R: 45.0
Total R: 10.0
Total R: 85.0
Total R: 75.0
Total R: 350.0
Elapsed time(s):  10
Total R: 95.0
Total R: 90.0
Total R: 135.0
Total R: 205.0
Total R: 105.0
Total R: 85.0
Total R: 150.0
Elapsed time(s):  20
Total R: 45.0
Total R: 80.0
Total R: 15.0
Total R: 425.0
Total R: 65.0
Total R: 35.0
Total R: 210.0
Elapsed time(s):  30
Total R: 110.0
Total R: 105.0
Total R: 230.0
Total R: 290.0
Total R: 215.0
Total R: 120.0
Total R: 80.0
Elapsed time(s):  40
Total R: 105.0
Total R: 50.0
Total R: 135.0
Total R: 135.0
Total R: 140.0
Total R: 190.0
Elapsed time(s):  50
Total R: 165.0
Total R: 180.0
Total R: 105.0
Total R: 55.0
Total R: 120.0
Total R: 110.0
Total R: 75.0
Total R: 210.0
Elapsed time(s):  60
Total R: 120.0
Total R: 160.0
Total R: 190.0
Total R: 370.0
Total R: 35.0
Total R: 410.0
Elapse

Total R: 155.0
Total R: 240.0
Total R: 50.0
Total R: 105.0
Elapsed time(s):  660
Total R: 120.0
Total R: 405.0
Total R: 425.0
Total R: 210.0
Total R: 100.0
Total R: 210.0
Total R: 110.0
Elapsed time(s):  670
Total R: 105.0
Total R: 75.0
Total R: 90.0
Total R: 320.0
Total R: 110.0
Elapsed time(s):  680
Total R: 155.0
Total R: 210.0
Total R: 155.0
Total R: 105.0
Total R: 115.0
Total R: 180.0
Total R: 155.0
Total R: 180.0
Elapsed time(s):  690
Total R: 210.0
Total R: 110.0
Total R: 75.0
Total R: 110.0
Total R: 30.0
Total R: 35.0
Total R: 55.0
Elapsed time(s):  700
Total R: 160.0
Total R: 180.0
Total R: 165.0
Total R: 135.0
Total R: 30.0
Total R: 75.0
Total R: 155.0
Elapsed time(s):  710
Total R: 120.0
Total R: 210.0
Total R: 155.0
Total R: 110.0
Total R: 30.0
Elapsed time(s):  720
Total R: 65.0
Total R: 75.0
Total R: 120.0
Total R: 155.0
Total R: 180.0
Total R: 15.0
Total R: 120.0
Total R: 50.0
Elapsed time(s):  730
Total R: 110.0
Total R: 75.0
Total R: 155.0
Total R: 30.0
Total R: 210.0


Total R: 55.0
Total R: 130.0
Total R: 110.0
Total R: 60.0
Total R: 75.0
Total R: 80.0
Total R: 125.0
Elapsed time(s):  1360
Total R: 55.0
Total R: 5.0
Total R: 155.0
Total R: 80.0
Total R: 105.0
Total R: 75.0
Elapsed time(s):  1370
Total R: 120.0
Total R: 305.0
Total R: 120.0
Total R: 90.0
Total R: 155.0
Total R: 35.0
Total R: 50.0
Total R: 105.0
Total R: 5.0
Total R: 80.0
Elapsed time(s):  1380
Total R: 105.0
Total R: 50.0
Total R: 30.0
Total R: 15.0
Total R: 45.0
Total R: 80.0
Total R: 410.0
Elapsed time(s):  1390
Total R: 105.0
Total R: 105.0
Total R: 50.0
Total R: 275.0
Total R: 65.0
Total R: 210.0
Total R: 15.0
Total R: 80.0
Elapsed time(s):  1400
Total R: 30.0
Total R: 210.0
Total R: 30.0
Total R: 110.0
Total R: 100.0
Total R: 20.0
Elapsed time(s):  1410
Total R: 30.0
Total R: 110.0
Total R: 170.0
Total R: 80.0
Total R: 260.0
Total R: 55.0
Total R: 140.0
Elapsed time(s):  1420
Total R: 260.0
Total R: 100.0
Total R: 20.0
Total R: 105.0
Elapsed time(s):  1430
Total R: 75.0
Total R:

In [3]:
#print(Reward_index)
env_test.run()
#env_test.close();

Total R: 0.0
Total R: 0.0


KeyboardInterrupt: 

In [ ]:
#Imports
import numpy as np
import os
import gym

import copy

from logging import getLogger
from gym import error, spaces
from gym import utils
from gym.utils import seeding

import chainer

from chainer import serializers

from chainer import functions as F


try:
    import atari_py
except ImportError as e:
    raise error.DependencyNotInstalled("{}. (HINT: you can install Atari dependencies by running 'pip install gym[atari]'.)".format(e))
    
def to_ram(ale):
    ram_size = ale.getRAMSize()
    ram = np.zeros((ram_size),dtype=np.uint8)
    ale.getRAM(ram)
    return ram    

In [ ]:
import time
num_episodes = 1
env = gym.make("SpaceInvaders-ram-v0")

for i in range(num_episodes):
        state = env.reset() #reset to initial state
        state = np.expand_dims(state, axis=0)/2
        terminal = False # reset terminal flag
        while not terminal:
            env.render()
            time.sleep(.05)
            q_values = model.get_q_values(state)
            policy = eps_greedy_policy(q_values.squeeze(), .1) # greedy policy
            action = np.random.choice(num_actions, p=policy)
            state, reward, terminal, _ = env.step(action) # take one step in the evironment
            state = np.expand_dims(state, axis=0)/2
# close window
env.close();

In [ ]:
import numpy as np
import gym
from keras.utils.np_utils import to_categorical as one_hot
from collections import namedtuple
from dqn_model import DoubleQLearningModel, ExperienceReplay

def train_loop_ddqn(model, env, num_episodes, batch_size=64, gamma=.94):        
    Transition = namedtuple("Transition", ["s", "a", "r", "next_s", "t"])
    eps = 1.
    eps_end = .1 
    eps_decay = .0005
    R_buffer = []
    R_avg = []
    for i in range(num_episodes):
        state = env.reset() #reset to initial state
        state = np.expand_dims(state, axis=0)/2
        terminal = False # reset terminal flag
        ep_reward = 0
        q_buffer = []
        steps = 0
        while not terminal:
            env.render() # comment this line out if jous' don't want to render the environment
            steps += 1
            q_values = model.get_q_values(state)
            q_buffer.append(q_values)
            policy = eps_greedy_policy(q_values.squeeze(), eps) 
            action = np.random.choice(num_actions, p=policy) # sample action from epsilon-greedy policy
            new_state, reward, terminal, _ = env.step(action) # take one step in the evironment
            new_state = np.expand_dims(new_state, axis=0)/2
            
            # only use the terminal flag for ending the episode and not for training
            # if the flag is set due to that the maximum amount of steps is reached 
            t_to_buffer = terminal if not steps == 200 else False
            
            # store data to replay buffer
            replay_buffer.add(Transition(s=state, a=action, r=reward, next_s=new_state, t=t_to_buffer))
            state = new_state
            ep_reward += reward
            
            # if buffer contains more than 1000 samples, perform one training step
            if replay_buffer.buffer_length > 1000:
                s, a, r, s_, t = replay_buffer.sample_minibatch(batch_size) # sample a minibatch of transitions
                q_1, q_2 = model.get_q_values_for_both_models(np.squeeze(s_))
                td_target = calculate_td_targets(q_1, q_2, r, t, gamma)
                model.update(s, td_target, a)    
                
        eps = max(eps - eps_decay, eps_end) # decrease epsilon        
        R_buffer.append(ep_reward)
        
        # running average of episodic rewards
        R_avg.append(.05 * R_buffer[i] + .95 * R_avg[i-1]) if i > 0 else R_avg.append(R_buffer[i])
        print('Episode: ', i, 'Reward:', ep_reward, 'Epsilon', eps, 'mean q', np.mean(np.array(q_buffer)))
        
        # if running average > 195, the task is considerd solved
    return R_buffer, R_avg

class AtariEnv(gym.Env, utils.EzPickle):
    metadata = {'render.modes': ['human', 'rgb_array']}

    def __init__(self, game='SpaceInvaders-v0', obs_type='ram', frameskip=(2, 5), repeat_action_probability=0.):
        """Frameskip should be either a tuple (indicating a random range to
        choose from, with the top value exclude), or an int."""

        utils.EzPickle.__init__(self, game, obs_type, frameskip, repeat_action_probability)
        assert obs_type in ('ram', 'image')

        self.game_path = atari_py.get_game_path(game)
        if not os.path.exists(self.game_path):
            raise IOError('You asked for game %s but path %s does not exist'%(game, self.game_path))
        self._obs_type = obs_type
        self.frameskip = frameskip
        self.ale = atari_py.ALEInterface()
        self.viewer = None

        # Tune (or disable) ALE's action repeat:
        # https://github.com/openai/gym/issues/349
        assert isinstance(repeat_action_probability, (float, int)), "Invalid repeat_action_probability: {!r}".format(repeat_action_probability)
        self.ale.setFloat('repeat_action_probability'.encode('utf-8'), repeat_action_probability)

        self.seed()

        self._action_set = self.ale.getMinimalActionSet()
        self.action_space = spaces.Discrete(len(self._action_set))

        (screen_width,screen_height) = self.ale.getScreenDims()
        if self._obs_type == 'ram':
            self.observation_space = spaces.Box(low=0, high=255, dtype=np.uint8, shape=(128,))
        elif self._obs_type == 'image':
            self.observation_space = spaces.Box(low=0, high=255, shape=(screen_height, screen_width, 3), dtype=np.uint8)
        else:
            raise error.Error('Unrecognized observation type: {}'.format(self._obs_type))

    def seed(self, seed=None):
        self.np_random, seed1 = seeding.np_random(seed)
        # Derive a random seed. This gets passed as a uint, but gets
        # checked as an int elsewhere, so we need to keep it below
        # 2**31.
        seed2 = seeding.hash_seed(seed1 + 1) % 2**31
        # Empirically, we need to seed before loading the ROM.
        self.ale.setInt(b'random_seed', seed2)
        self.ale.loadROM(self.game_path)
        return [seed1, seed2]

    def step(self, a):
        reward = 0.0
        action = self._action_set[a]

        if isinstance(self.frameskip, int):
            num_steps = self.frameskip
        else:
            num_steps = self.np_random.randint(self.frameskip[0], self.frameskip[1])
        for _ in range(num_steps):
            reward += self.ale.act(action)
        ob = self._get_obs()

        return ob, reward, self.ale.game_over(), {"ale.lives": self.ale.lives()}

    def _get_image(self):
        return self.ale.getScreenRGB2()

    def _get_ram(self):
        return to_ram(self.ale)

    @property
    def _n_actions(self):
        return len(self._action_set)

    def _get_obs(self):
        if self._obs_type == 'ram':
            return self._get_ram()
        elif self._obs_type == 'image':
            img = self._get_image()
        return img

    # return: (states, observations)
    def reset(self):
        self.ale.reset_game()
        return self._get_obs()

    def render(self, mode='human'):
        img = self._get_image()
        if mode == 'rgb_array':
            return img
        elif mode == 'human':
            from gym.envs.classic_control import rendering
            if self.viewer is None:
                self.viewer = rendering.SimpleImageViewer()
            self.viewer.imshow(img)
            return self.viewer.isopen

    def close(self):
        if self.viewer is not None:
            self.viewer.close()
            self.viewer = None

    def get_action_meanings(self):
        return [ACTION_MEANING[i] for i in self._action_set]

    def get_keys_to_action(self):
        KEYWORD_TO_KEY = {
            'UP':      ord('w'),
            'DOWN':    ord('s'),
            'LEFT':    ord('a'),
            'RIGHT':   ord('d'),
            'FIRE':    ord(' '),
        }

        keys_to_action = {}

        for action_id, action_meaning in enumerate(self.get_action_meanings()):
            keys = []
            for keyword, key in KEYWORD_TO_KEY.items():
                if keyword in action_meaning:
                    keys.append(key)
            keys = tuple(sorted(keys))

            assert keys not in keys_to_action
            keys_to_action[keys] = action_id

        return keys_to_action

    def clone_state(self):
        """Clone emulator state w/o system state. Restoring this state will
        *not* give an identical environment. For complete cloning and restoring
        of the full state, see `{clone,restore}_full_state()`."""
        state_ref = self.ale.cloneState()
        state = self.ale.encodeState(state_ref)
        self.ale.deleteState(state_ref)
        return state

    def restore_state(self, state):
        """Restore emulator state w/o system state."""
        state_ref = self.ale.decodeState(state)
        self.ale.restoreState(state_ref)
        self.ale.deleteState(state_ref)

    def clone_full_state(self):
        """Clone emulator state w/ system state including pseudorandomness.
        Restoring this state will give an identical environment."""
        state_ref = self.ale.cloneSystemState()
        state = self.ale.encodeState(state_ref)
        self.ale.deleteState(state_ref)
        return state

    def restore_full_state(self, state):
        """Restore emulator state w/ system state including pseudorandomness."""
        state_ref = self.ale.decodeState(state)
        self.ale.restoreSystemState(state_ref)
        self.ale.deleteState(state_ref)

ACTION_MEANING = {
    0 : "NOOP",
    1 : "FIRE",
    2 : "UP",
    3 : "RIGHT",
    4 : "LEFT",
    5 : "DOWN",
    6 : "UPRIGHT",
    7 : "UPLEFT",
    8 : "DOWNRIGHT",
    9 : "DOWNLEFT",
    10 : "UPFIRE",
    11 : "RIGHTFIRE",
    12 : "LEFTFIRE",
    13 : "DOWNFIRE",
    14 : "UPRIGHTFIRE",
    15 : "UPLEFTFIRE",
    16 : "DOWNRIGHTFIRE",
    17 : "DOWNLEFTFIRE",
}